# JupyterLite

## WASM（WebAssembly）の概要

**WebAssembly（WASM）** は、ブラウザ上で高速に動作するバイナリフォーマットの実行環境です。
従来、Webブラウザで実行可能な言語はJavaScriptに限られていましたが、WASMを利用することで、C、C++、Pythonなどのコードをコンパイルしてブラウザ上で実行できるようになりました。

WASMは次のような特徴を持ちます。

* **高速性**：ネイティブコードに近い速度で実行可能です。
* **安全性**：ブラウザのサンドボックス環境内で安全に動作します。
* **移植性**：主要なブラウザ（Chrome、Firefox、Safari、Edge）で動作します。

この技術を基盤として、Pythonをブラウザ上で実行できる環境が **Pyodide** です。

https://ruoyu0088.github.io/jupyterlite/lab/index.html

## Pyodideの紹介

**Pyodide** は、CPythonインタプリタをWebAssembly上に移植したプロジェクトです。
これにより、ブラウザ内でPythonを実行でき、NumPy、Matplotlib、Pandasといった主要な科学計算ライブラリも利用できます。

Pyodideは内部的にはEmscriptenによってビルドされており、WebAssembly形式のバイナリとして提供されます。
ブラウザがPyodideをロードすると、JavaScript経由でPythonコードを実行することが可能になります。

### Pyodideの基本的な使い方

Pyodideを直接ブラウザのコンソールやJavaScriptから操作することもできます。
次の例では、Pyodideを初期化し、Pythonコードを実行しています。Pyodideでは、JavaScriptのオブジェクトをPython側で操作したり、逆にPythonのデータをJavaScript側で利用することも可能です。

In [1]:
%%writefile test_pyodide.html
<!doctype html>
<html>
  <head>
      <script src="https://cdn.jsdelivr.net/pyodide/v0.29.0/full/pyodide.js"></script>
  </head>
  <body>
    Pyodide test page <br>
    Open your browser console to see Pyodide output
    <script type="text/javascript">
      async function main(){
        let pyodide = await loadPyodide();
        console.log(pyodide.runPython(`
            import sys
            sys.version
        `));
        pyodide.runPython("print(1 + 2)");
      }
      main();
    </script>
  </body>
</html>

Writing test_pyodide.html


## JupyterLiteの紹介

**JupyterLite** は、JupyterLabをWebAssembly（Pyodide）上で動作させるプロジェクトです。
従来のJupyterLabとは異なり、サーバーを必要とせず、すべての処理がブラウザ内で完結します。

JupyterLiteは次の要素から構成されます。

* **Pyodideカーネル**：Pythonコードの実行を担当
* **JupyterLabフロントエンド**：Notebook UIを提供
* **Service Worker / IndexedDB**：ローカルストレージやファイルの永続化を実現

In [ ]:
import js
import pyodide
import pyodide_js

pyodide.code.run_js("""
add = function(x, y){ return x + y;}
""")

js.add(1, 2)

![](./benchmark.png)

### JupyterLiteの構造

JupyterLiteの内部構造は次のようになっています。

```
jupyterlite/
 ├── app/                # JupyterLiteアプリ本体
 ├── extensions/         # カスタム拡張機能
 ├── lite.json           # 設定ファイル
 ├── pyodide/            # Pyodide関連のビルド成果物
 └── build/              # ビルド結果（静的ファイル）
```

### WorkerでPyodideを実行する仕組み

ブラウザ上でPyodideを直接実行することも可能ですが、重い処理を行う場合はメインスレッドがブロックされてしまいます。
そのため、一般的には **Web Worker** を利用してPyodideをバックグラウンドスレッドで実行します。

これにより、ブラウザ上で完全にローカルにPythonコードが実行され、結果が返されます。
インターネット上のPythonサーバーに依存しない点が、JupyterLiteの基盤となる仕組みです。

## JupyterLiteのビルド方法

JupyterLiteは`pip`でインストールしてビルドできます。
以下の手順でローカル環境にJupyterLiteを構築できます。

```python
# JupyterLiteのインストール
!pip install jupyterlite-core jupyterlite-pyodide-kernel

# サンプルノートブックの配置
!mkdir content
!echo "print('Hello JupyterLite')" > content/example.py

# ビルド実行
!jupyter lite build --contents content
```

このコマンドを実行すると、`_output/` ディレクトリ内に静的なJupyterLiteサイトが生成されます。

## GitHub Pagesへのデプロイ

GitHub Pagesにデプロイするには、ビルドした静的ファイルをリポジトリにコミットし、Pagesを有効化します。

```python
# gh-pagesブランチにデプロイ
!jupyter lite build --output-dir dist
!git init
!git checkout -b gh-pages
!cp -r dist/* .
!git add .
!git commit -m "Deploy JupyterLite"
!git push -f origin gh-pages
```

GitHubのリポジトリ設定で「Pages」のソースを `gh-pages` ブランチに設定すれば、
JupyterLiteがブラウザ上で動作する完全なJupyter環境として利用できます。